# Robust Test - Median Noise - columns
for each column in the data we will permute the values by setting randomly values with diffrent noise  around the median value.

# Import

In [1]:
import pandas as pd
import numpy as np
from pycaret.classification import *
import time
# from DB_scripts import rnd_bln_split_CSV as shf

# Settings

In [2]:
# set constants
target_label = 'tuple'
learning_model = 'rf' # ['rf','et','lightgbm','xgboost']
num_features = ['min_packet_size', 'min_fpkt', 'min_bpkt']
file_name = "new_all_features_"
path = "../Datasets/" + target_label + "_dataset/"

In [3]:
# function for making model-prediction over the data set and measure the run time 
def timed_prediction(in_data,in_model):
    t = time.process_time()
    predicted = predict_model(in_model, data=in_data)
    elapsed_time = time.process_time() - t
    print("prediction took: " + str(elapsed_time))
    return predicted

In [4]:
# compare answers and labeled test
def compare_prediction_with_answers(in_predicted, in_answers):
    count=0
    index = in_predicted.index
    number_of_rows = len(index)
    errors_arr = []
    for i in range(0,number_of_rows):
        if str(int(in_answers.iloc[i])) != str(int(in_predicted.iloc[i]['Label'])):
            count=count+1
            cur_error = str(in_answers[i]) + "!=" + str(in_predicted.iloc[i]['Label'])
            errors_arr.append(cur_error)
#             print("error in line " + str(i) +
#                   " " + str(in_answers[i]) +
#                   "!=" + str(in_predicted.iloc[i]['Label']))
#     print("Errors: " + str(errors_arr))
#     print("Number of error: " + str(count) + " from " +
#           str(number_of_rows) + " test samples \nWhich is "
#           + str(100*count/number_of_rows) + "% of error.")
    return count

In [5]:
# function for checkign the correction of the model-prediction over the data
def check_correction(in_predicted):
    in_answers=in_predicted['Label']
    return compare_prediction_with_answers(in_predicted, in_answers)

In [6]:
# activating balanced random data shuffling
# shf.split_CSV_randomly_balanced(target_label,file_name)

In [7]:
def get_rand_dataframe(in_data, in_column_indx,in_noise):
    cur_col=in_data[in_column_indx]
    col_len = len(cur_col)
    max_value = cur_col.max()
    value = cur_col[0]
    min_value = cur_col.min()
    noise = in_noise
    buffer = cur_col.median()*noise
    for i in range(0,col_len):
        cur_val = cur_col[i]
        coeff = np.random.randint(0,1)
        if coeff == 0: coeff=-1
        cur_col[i] = cur_val + (coeff * buffer)
    in_data[in_column_indx] = cur_col
    return in_data

# Read Data

In [8]:
data = pd.read_csv(path+r'new_all_features_'+target_label+'_train.csv',
                      sep='\t',
                      skiprows=[1])

# Setup Data and Build Model

In [9]:
setup(data=data,
      target=target_label,
      numeric_features=num_features,
      silent=True)
model=create_model(learning_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9654,0.0000,0.8800,0.9643,0.9642,0.9594,0.9595
1,0.9703,0.0000,0.9063,0.9715,0.9703,0.9653,0.9653
2,0.9594,0.0000,0.8496,0.9600,0.9580,0.9525,0.9526
3,0.9634,0.0000,0.8363,0.9663,0.9633,0.9572,0.9572
4,0.9733,0.0000,0.8851,0.9726,0.9723,0.9688,0.9688
5,0.9614,0.0000,0.8339,0.9617,0.9603,0.9548,0.9549
6,0.9693,0.0000,0.8919,0.9684,0.9677,0.9641,0.9642
7,0.9644,0.0000,0.8521,0.9626,0.9622,0.9583,0.9584
8,0.9703,0.0000,0.8699,0.9691,0.9682,0.9653,0.9653
9,0.9673,0.0000,0.8525,0.9674,0.9658,0.9617,0.9618


### Read Test Data

In [10]:
unseen_data = pd.read_csv(path+file_name+target_label+'_test.csv',
                      sep='\t',
                      skiprows=[1])

### Predicte and Check

In [11]:
answers = unseen_data[target_label]
unseen_data = unseen_data.drop(columns=[target_label])

In [12]:
predicted = predict_model(model, data=unseen_data)

### number of errors of the model for current test set

In [13]:
compare_prediction_with_answers(predicted,answers)

199

# Add noise and run test
each round noise will be increased in 5%

In [37]:
noise=0.05
results = []
results_map = []
for index in range(0,4):
    ### for any clolumn in the data set
    for cur_col in data.columns:
        ### Read Test Data
        unseen_data = pd.read_csv(path+file_name+target_label+'_test.csv',
                          sep='\t',
                          skiprows=[1])
        ### Change column values
        unseen_data = get_rand_dataframe(unseen_data,cur_col,noise)
        ### Predicte and Check
        answers = unseen_data[target_label]
        unseen_data = unseen_data.drop(columns=[target_label])
        predicted = predict_model(model, data=unseen_data)
        results.append(compare_prediction_with_answers(predicted,answers))

    ### creaing map of results and column names for printing
    count = 0
    results_map.append({})
    for cur_col in data.columns:
        results_map[index][cur_col] = results[count]
        count=count+1
    noise = noise+0.05

In [38]:
sorted_array = []
for index in range(0,4):
    sorted_array.append([(k, v) for k, v in sorted(results_map[0].items(), key=lambda x: x[1], reverse=True)])
#     ### prints results map sorted by values
#     #noise = noise - 0.05
#     #print("noise level = " + str(noise))
for index in range(0,4):
    noise=noise-0.05
    print("sorted features errors with noise level: " + str(noise) + "\n")
    print(sorted_array[index])
    print("\n")

sorted features errors with noise level: 0.2

[('tuple', 6189), ('SYN_tcp_winsize', 2065), ('ssl_v', 913), ('fcipher_suites', 699), ('SYN_tcp_scale', 274), ('fSSL_num_extensions', 244), ('size_histogram_3', 204), ('bpeak_features_4', 204), ('mean_biat', 204), ('max_fpkt', 204), ('size_histogram_9', 203), ('bpeak_features_8', 203), ('packet_count', 203), ('mean_packet_size', 203), ('bbytes', 203), ('std_fpkt', 203), ('mean_fttl', 203), ('size_histogram_5', 201), ('size_histogram_6', 201), ('fpeak_features_6', 201), ('bpeak_features_6', 201), ('sizevar', 201), ('bpackets', 201), ('fbytes', 201), ('std_biat', 201), ('size_histogram_8', 200), ('fpeak_features_3', 200), ('fpeak_features_4', 200), ('bpeak_features_5', 200), ('min_packet_size', 200), ('max_packet_size', 200), ('std_fiat', 200), ('max_biat', 200), ('fSSL_session_id_len', 199), ('size_histogram_7', 199), ('size_histogram_10', 199), ('fpeak_features_1', 199), ('fpeak_features_2', 199), ('fpeak_features_9', 199), ('bpeak_features

obviously the tuple feature is the most sensative one, but it is the predicted feature so just ignore it.
we can see that the Window-Size, SSL version and Cipher-suite features (each one represented with one sigle column in our dataset) are the most effective (maximum errors) features when noise added.